<h1>Mining with FCA</h1>
<p>This class explore data using Formal Concept Analysis, thus is important to have data as a formal context</p>

In [31]:
from itertools import chain, combinations
import ast



class FCAsequence():
    def __init__(self,context):
        self.context = context
        
    def powerset(self, iterable):
        s = list(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(len(s), -1, -1))

    def closure(self, objects): # cette fonction a pour objectif de retourner les objects qui possedent
                                # des attributs communs(les attributes communs inclu dans context) 
        attributes = set.intersection(*(self.context[obj] for obj in objects))  
        extent = {obj for obj in self.context if self.context[obj].issuperset(attributes)} # ce ligne est tres important
                                        # il permet de retourner tout les objets qui contenant les attributs
                                        # 'attributs' c-a-d l'ensemble des attributs le plus grand qui 
        return extent, attributes       # contient les attributs voir l'exemple de [9] et [0,4]
                                        # produit des concepts fermés
    def IncludeIn(self,I=tuple,L=tuple,S1=set,S2=set):
        if I[0]>=L[0] and I[1]<=L[1] and S1.issubset(S2) :
            return True
        else:
            return False
    def isSubCara(self,cara1,cara2):
        for c2 in set(cara2):
            isRedondant=True
            for c1 in set(cara1):
                I,S = c1.split(":")[0],c1.split(":")[1]
                Is,Ss = c2.split(":")[0],c2.split(":")[1]
                I,S=ast.literal_eval(I),ast.literal_eval(S)
                Is,Ss=ast.literal_eval(Is),ast.literal_eval(Ss)
                if self.IncludeIn(I,Is,S,Ss):
                    isRedondant=True
                else:
                    isRedondant=False
                    break
            if isRedondant==True:
                return True
        return False
    def isRedondant(self,d,e):
        g,c=e   #group and characteristic  # d is the description
        g,c= set(g),set(c)
        for gs,cs in d:
            if g.issubset(set(gs)): # ou niveau des individus
                if self.isSubCara(c,cs):
                    return True
        return False
    def brushCaract(self,fset):
        intervalles = list(fset)
        result = frozenset()

        while intervalles:
            intervalle = intervalles.pop()
            merged = set([intervalle])

            # Extraire les bornes et les caractéristiques
            bornes, caracteristique_str = intervalle.split(":")
            borne_inf, borne_sup = map(eval, bornes.strip("()").split(","))

            # Convertir la caractéristique en ensemble
            caracteristique = set(caracteristique_str.strip("{}").split(","))

            for autre in intervalles[:]:
                autres_bornes, autres_caracteristique_str = autre.split(":")
                autres_borne_inf, autres_borne_sup = map(eval, autres_bornes.strip("()").split(","))
                autres_caracteristique = set(autres_caracteristique_str.strip("{}").split(","))

                # Vérification de la fusion
                if caracteristique == autres_caracteristique:
                    if borne_inf <= autres_borne_sup and borne_sup >= autres_borne_inf:
                        merged.add(autre)
                        intervalles.remove(autre)

            # Ajouter le résultat fusionné
            if len(merged) > 1:
                min_borne = min(borne_inf, autres_borne_inf)
                max_borne = max(borne_sup, autres_borne_sup)
                result = result.union(frozenset({f"({min_borne}, {max_borne}):{{{','.join(caracteristique)}}}" }))
            else:
                result = result.union(merged)

        return frozenset(result)
    
    def generate_concepts(self):
        objects = list(self.context.keys())
        all_concepts = set()
        for obj_subset in self.powerset(objects):
#             print(obj_subset)
            if obj_subset:
                extent, intent = self.closure(obj_subset)
                concept = (frozenset(extent), self.brushCaract(frozenset(intent)))
#                 print(concept," not In\n",all_concepts,"\n___________")
                if self.isRedondant(all_concepts,concept)==False:
                    all_concepts.add(concept)
#                 else:
#                     print(concept)
        return sorted(all_concepts, key=lambda x: (len(x[1]), len(x[0])))


<h1>Sample data test</h1>

In [398]:
data={
    "person1":{(1,2):{"A","A1"},   (5,5.5):{"B"}, (6,8):{"C"}, (10,11):{"D"}, (13,13.9):{"E","E2"},  (16.99,17):{"F","G"}},
    "person2":{(1.5,2):{"A","A1"}, (6,7):{"B1"},   (7,8):{"C"}, (10.5,11):{"D1"}, (13,14):{"E","E2"},    (14.99,15):{"F","G"}},
    "person3":{(1,2):{"AA","AA1"}, (4,5.5):{"B1"}, (6,8.4):{"C1"}, (9,11):{"D"}, (12,13.9):{"EE"}, (15.99,16):{"F","G","H"}},
    "person4":{(1,1.5):{"A","A2"}, (2,4.7):{"B2"}, (5.6,8):{"C2"}, (10,11):{"D1"}, (11,14):{"E","E1"},    (16.99,18):{"F","E"}},
    "person5":{(0,2):{"A","A2"},   (2,3):{"B2"}, (6,9):{"C1"}, (10,11):{"DD"}, (13,13.9):{"E","E1"},  (16.99,19):{"F","A"}},
    "person6":{(0,2):{"AA"},       (3,4.5):{"B3"}, (6,8):{"C1"}, (10,12):{"D2"}, (13,14):{"E"},    (14.99,15):{"F","D"}},
    "person7":{(1,3):{"A","A1"},   (4,6.5):{"B4"}, (7,8):{"C2"}, (9,11):{"D"}, (11.5,13.9):{"E","E1"},(13.99,15):{"F","G"}},
}


In [35]:
data={
    "person_vert":{(8,9):(((7.5,0.5),(8.5,1.5)),{"T"}),   (10,11):(((9,1.5),(10,1.5)),{"C"}), (14,15):(((10,2),(10,3)),{"T"}), (16,18):(((10,3.5),(9.5,5)),{"T","S"}), (20,21):(((9.5,5.5),(8.5,7)),{"R"})},
    "person_rouge":{(8,9.5):(((6,2.5),(7,3.5)),{"T"}), (10,12):(((7.5,4),(7.5,5)),{"T","C"}),   (13,14):(((7.5,5.5),(6.5,6.5)),{"M"}), (14.5,15):(((6,6.5),(4.5,6.5)),{"T"}), (16,17):(((4,6.5),(3,5.5)),{"T","S"}),(17.5,18):(((2,5),(1,4)),{"S"}),(18,19):(((1.5,3.5),(1.5,2)),{"S","M"}),(20,21):(((2,1.5),(1,0.5)),{"M"})},         
    "person_bleu":{(7,9):(((0.5,0.5),(1.5,1.5)),{"M","T"}), (9.2,10):(((2,2),(3,2.5)),{"T"}), (10.1,12):(((3.5,3),(4.5,4)),{"C"}), (13,15):(((5.5,4.5),(6.5,5.5)),{"M","T"}), (15.5,16):(((7,5),(7,4)),{"T"}), (17,18):(((6.5,3.5),(5.5,2.5)),{"S"})},
    "person_noir":{(8.5,9):(((1,0.5),(1,1.5)),{"T"}), (9.5,10):(((1,2),(2,3)),{"T","C"}), (11,12):(((2.5,3.5),(4.5,3.5)),{"C","T"}), (13,14):(((5,3.5),(5,5)),{"M","T"}), (14.5,15):(((5.5,5),(6.5,6)),{"T"}),    (15,16):(((6,6),(5,6)),{"T"})},
    "person_noir122":{(8.5,9):(((90,5),(90,5.5)),{"T"})},
    "person_noir123":{(8.5,9):(((90,5),(90,5.5)),{"T"})},
    
}

<h1>Build the relevant formal context</h1>
use a set of sequences like what is in above and try to extract from it the formal context following these steps :

In [135]:
import math
from itertools import chain, combinations
import pandas as pd


class Spatio_temporal_sequences():
    def __init__(self,data,cardinality,treshold_meter=0):
        self.data=data
        self.cardinality=cardinality
        self.treshold_meter=treshold_meter
        self.memberes = list(data.keys())
    
    
    def Haversine_distance(self,point1, point2): #distance orthodromique (ou formule de Haversine)
        lat1, lon1=point1
        lat2, lon2=point2

        # Rayon de la Terre en kilomètres
        r = 6371.0

        # Conversion des degrés en radians
        lat1 = math.radians(lat1)
        lon1 = math.radians(lon1)
        lat2 = math.radians(lat2)
        lon2 = math.radians(lon2)

        # Différences de latitude et de longitude
        dlat = lat2 - lat1
        dlon = lon2 - lon1

        # Formule de Haversine
        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

        # Distance en metres
        distance = r * c
        return distance*1000

    def euclidean_distance(self,a,b):
        lat1, lon1=a
        lat2, lon2=b
        return math.sqrt((lat2-lat1)**2+(lon2-lon1)**2)
    
    def intersection_intervals(self,interval1, interval2):
        start = max(interval1[0], interval2[0])
        end = min(interval1[1], interval2[1])

        if start < end:
            return (start, end)
        else:
            return None
    def transform_data_if_needed(self,inner_dict):
        #This function enhance this data structure "(8.5, 9): [(((0.75, 0.5), (1.25, 1.5)), {'T'})]" 
        # which is the output of the 'commun_sequences' to have the characteristic form without [] marks.
        
        new_inner_dict = {}

        for inner_key, value in inner_dict.items():
            combined_set = set()
            associated_tuples = []

            # Normaliser la structure en une liste de tuples pour traitement uniforme
            values_to_process = value if isinstance(value, list) else [value]

            for item in values_to_process:
                if isinstance(item, tuple) and len(item) == 2:
                    associated_tuples.append(item[0])
                    combined_set.update(item[1])

            new_inner_dict[inner_key] = (tuple(associated_tuples[0]), combined_set)

        return new_inner_dict
    def isAfterInterval(self,interval1,interval2):
        x,y=interval1
        x1,y1=interval2
        if x1>y:
            return True
        else:
            return False
    def commun_sequences(self,sequence1, sequence2,id1,id2=list,treshold_meter=0):            
        sequence2 = self.transform_data_if_needed(sequence2)
        ids={id1,*id2}
        idsl=list(ids)
        idsl.sort()
        matched_person={tuple(idsl):{}}
        sequence_commun={}
        for interval1 in sequence1:
            for interval2 in sequence2:
                if self.isAfterInterval(interval1=interval1,interval2=interval2)==True:
                    break
                if self.intersection_intervals(interval1, interval2):
    #                 print(sequence1[interval1][1])
                    if (self.Haversine_distance(sequence1[interval1][0][0],sequence2[interval2][0][0])+self.Haversine_distance(sequence1[interval1][0][1],sequence2[interval2][0][1]))<=treshold_meter:
    #                     print(sequence1[interval1][1])
                        if sequence1[interval1][1] & sequence2[interval2][1]: #ici le travail
                            if len(list(sequence1[interval1][1] & sequence2[interval2][1]))>1:
                                sequence_commun[self.intersection_intervals(interval1, interval2)]=list()
                                for elm in list(sequence1[interval1][1] & sequence2[interval2][1]):
                                    sequence_commun[self.intersection_intervals(interval1, interval2)].append(
                                                                                                            ( ( ((sequence1[interval1][0][0][0]+sequence2[interval2][0][0][0])/2,(sequence1[interval1][0][0][1]+sequence2[interval2][0][0][1])/2),
                                                                                                                (((sequence1[interval1][0][1][0]+sequence2[interval2][0][1][0])/2,(sequence1[interval1][0][1][1]+sequence2[interval2][0][1][1])/2))   
                                                                                                              ),
                                                                                                              {elm}
                                                                                                            )
                                                                                                        )         
                            else:
                                sequence_commun[self.intersection_intervals(interval1, interval2)]=[( ( ((sequence1[interval1][0][0][0]+sequence2[interval2][0][0][0])/2,(sequence1[interval1][0][0][1]+sequence2[interval2][0][0][1])/2),
                                                                                                                (((sequence1[interval1][0][1][0]+sequence2[interval2][0][1][0])/2,(sequence1[interval1][0][1][1]+sequence2[interval2][0][1][1])/2))   
                                                                                                              ),
                                                                                                              sequence1[interval1][1] & sequence2[interval2][1]
                                                                                                            )]   
                            idsl=list(ids)
                            idsl.sort()
                            matched_person[tuple(idsl)]= sequence_commun
        if len(list(matched_person[tuple(idsl)])) ==0:
            return None
        else:
            return matched_person
    
    def powerset(self,iterable): 
        s = list(iterable)         
        return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
    def getGroupsMore_N_element(self,cardinality):#pour creer des combinison des cles de sequence
        data=self.data
        groups={}
        for i in range(cardinality,len(list(data.keys()))+1): #initialisation de dictionnaire des combinaisons (cle:[])
            groups[len(list(data.keys()))+cardinality-i]=list() # sachant que la cle represent la cardinalite des groupes
                                              # c-a-d les groupes des listes de 2 elements de 3,4 etc

        for obj in self.powerset(list(data.keys())):
            if len(list(obj))>=cardinality:
                groups[len(list(obj))].append(set(obj))
        return groups
    def getCommunsequences(self):
        treshold=self.treshold_meter
        data=self.data
        groups=self.getGroupsMore_N_element(cardinality=self.cardinality)
        results={}
        for keyGroup in groups:
            results[keyGroup]=list()

        for keyGroup in groups: 
            for group in groups[keyGroup]:
                liste=list(group)
                liste.sort()
                result=None
                result =self.commun_sequences(data[liste[0]],data[liste[1]],liste[0],[liste[1]],treshold_meter=treshold) #common subsequence between first and second sequence           
                for i in range(1,len(liste)):
                    if result!=None and liste[i] not in list(list(result)[0]): # le 2eme condi pour le cas de repetition de comparaison 
    #                     print(list(result)[0],liste[i])
                        result =self.commun_sequences(data[liste[i]],result[list(result)[0]],liste[i],list(list(result)[0]),treshold_meter=treshold)
                        if result==None:
                            break
                if result :
                    results[keyGroup].append(result)

        deleted_keys=[]
        for key in results:
            if len(results[key])==0:
                deleted_keys.append(key)
        for key in deleted_keys:
            del results[key]
        return results
    #While this comment here so this function is not used and we will discuss about it.
    def dropLocationPart(self):
        clusters=self.getCommunsequences()
        new_clusters=dict()
        for cardinality in clusters:
            new_clusters[cardinality]=list()
            group_key=dict()
    #         print(cardinality)
            for group in clusters[cardinality]:
                group_key[list(group.keys())[0]]=dict()
    #             print(list(group.keys())[0])
                for key in group:
                    for interval_time in group[key]:
                        for item in group[key][interval_time]:
                            group_key[key][interval_time]=[item[1]]
            new_clusters[cardinality].append(group_key)
        return new_clusters
    
    def replaceLocationPart(self):
        new_attributes={}
        for cardi in commun_sequences:
            new_attributes[cardi]=[]

        groupes={}    
        for cardi in commun_sequences:
            for sequence in commun_sequences[cardi]:
                for key in sequence:
                    groupes[key]={}
        positions={}
        i=0
        for cardi in commun_sequences:
            for sequence in commun_sequences[cardi]:
                for key in sequence:
                    for interval in sequence[key]:
                        for location_activity in sequence[key][interval]:
        #                     print(i,location_activity[0]) 
                            positions[f"P{i}"]=location_activity[0]
                            i+=1
        #                     groupes[key].append({interval:location_activity})
        for cardi in commun_sequences:
            for sequence in commun_sequences[cardi]:
                for key in sequence:
                    groupe_key={}
                    for interval in sequence[key]:
                        for location_activity in sequence[key][interval]:
                            for position_key in positions:
                                if sts.Haversine_distance(positions[position_key][0],location_activity[0][0])+sts.Haversine_distance(positions[position_key][1],location_activity[0][1])<5000000:         
                                    activity = list(location_activity[1])
                                    activity.append(position_key)
                                    groupes[key][interval]=[set(activity)]
                                    groupe_key[key]=groupes[key]
#                                     print(position_key)
                                    break
                    new_attributes[cardi].append(groupe_key)
        return new_attributes

    
    def getAttributs(self): #extract attributs from clusters of groups and commun subsequences to
                            #create context
#         clusters=self.dropLocationPart()
        clusters=self.replaceLocationPart()
#         print(clusters[3])
        
        attributes=[]               
        for cardinality in clusters:
            for group in clusters[cardinality]:
                for key in group:
                    for item in group[key]:
    #                     print(item,"-",group[key][item])
    #                     gk = list(item)[0] # time interval
    #                     if {gk:group[key][gk][0]} not in attributes:
    #                     if {item:group[key][item]} not in attributes:
                        if len(group[key][item])>1: #cette condition peux creer des repititions on va les elminer avec un autre condition        
                            for el in group[key][item]:
                                if {item:el} not in attributes:
                                    attributes.append({item:el})
                        else:
                            if {item:group[key][item][0]} not in attributes:
                                attributes.append({item:group[key][item][0]})

        return attributes
    def getContext_table(self):
        memberes= list(data.keys())
        resultats=self.replaceLocationPart()
        caracteristiques=self.getAttributs()
        columns=[]
        for caracteristique in caracteristiques:
            for item in caracteristique:
                columns.append(f"{item}:{caracteristique[item]}")
        df = pd.DataFrame(0, index=memberes, columns=columns)

        # Remplir le DataFrame
        for cardi in resultats:
            for item in resultats[cardi]:
                for groupe, caract in item.items():
                    for member in groupe:
                        for car in caract:
        #                     print(f"{car}:{caract[car]}")
                            if len(caract[car])>=2:
                                for e in caract[car]:
#                                     print(f"{car}:{e,member}")
                                    col_name = f"{car}:{e}"
        #                             print(col_name)
                                    df.at[member, col_name] = 1
                            else:
                                col_name = f"{car}:{caract[car][0]}"
                                df.at[member, col_name] = 1
        return df
    def getContext(self):
        Binary_dataframe=self.getContext_table()
        context = dict()
        for index,row in Binary_dataframe.iterrows():
            attributs = []
            for i in range(len(row)):
                if row[i]==1:
                    attributs.append(list(Binary_dataframe.columns)[i])
            context[index]=set(attributs)    
        return context
    def get_liste_interval(self,description):
        liste_intervals=[]
        for item in description:
            liste_intervals.append(ast.literal_eval(item.split(":")[0]))
        return liste_intervals
    def moyenne_coordonnees(self,coordonnees):
        # Initialiser les variables pour stocker la somme des coordonnées et le nombre de points
        somme_x = 0
        somme_y = 0
        nombre_points = 0

        # Parcourir chaque paire de coordonnées et chaque point
        for paire in coordonnees:
            for point in paire:
                somme_x += point[0]
                somme_y += point[1]
                nombre_points += 1

        # Calculer les moyennes
        moyenne_x = somme_x / nombre_points
        moyenne_y = somme_y / nombre_points

        return moyenne_x, moyenne_y
    def get_centers_locations(self,sorted_results):
        data=self.data
        concepts=dict()
        for intent, extent in sorted_results:
            if intent:
                concepts[tuple(extent)]=list()
                for interval_selected in self.get_liste_interval(intent):
                    liste_person_location=list()
                    for individual in data:
                        for interval in data[individual]:
                            inter = self.intersection_intervals(interval,interval_selected)
                            if inter and individual in extent:
                                liste_person_location.append((inter,individual,data[individual][interval][0]))
                    concepts[tuple(extent)].append(liste_person_location)

            else:
                concepts[tuple(extent)]=None  
        person_location_centers=dict()

        for group in concepts:
            if concepts[group]:
                person_location_centers[group]=list()
                for subgroup in concepts[group]:
                    subgroup_locations=dict()
                    liste_locations=[]
                    for triple in subgroup:
                        key_time=triple[0]
                        subgroup_locations[key_time]=None
                        liste_locations.append(triple[2])
                    subgroup_locations[key_time]=self.moyenne_coordonnees(liste_locations)
                    person_location_centers[group].append(subgroup_locations)
            else:
                person_location_centers[group]=None
        liste_time=[]
        for group in person_location_centers:
            liste_time.append(person_location_centers[group])
        return liste_time
    def sort_result(self,concept):
        results = []

        for extent, intent in concept:
            results.append((set(intent), set(extent)))

        # Trier les résultats par le nombre d'individus (taille de l'ensemble d'individus) en ordre décroissant
        sorted_results = sorted(results, key=lambda x: len(x[1]), reverse=True)
        return sorted_results
    def build_data_frame_result(self,concept):
        sorted_results=self.sort_result(concept)
        intents=[]
        extents=[]
        # # Afficher les résultats triés
        # for intent, extent in sorted_results:
        #     print(f"Predicats: {intent} \n Individuals: {extent}\n")
        for intent, extent in sorted_results:
            intents.append(intent)
            extents.append(extent)
        concept_table=pd.DataFrame()
        concept_table["Extent"]=extents
        concept_table["Intent"]=intents
        return concept_table



In [140]:
sts = Spatio_temporal_sequences(data=data,cardinality=2,treshold_meter=5000000)
cs = sts.commun_sequences(data["person_bleu"],data["person_noir"],"person_bleu",["person_noir"],treshold_meter=5000000)   
cs

{('person_bleu',
  'person_noir'): {(8.5, 9): [(((0.75, 0.5), (1.25, 1.5)), {'T'})], (9.5,
   10): [(((1.5, 2.0), (2.5, 2.75)), {'T'})], (11,
   12): [(((3.0, 3.25), (4.5, 3.75)), {'C'})], (13,
   14): [(((5.25, 4.0), (5.75, 5.25)), {'T'}),
   (((5.25, 4.0), (5.75, 5.25)), {'M'})], (14.5,
   15): [(((5.5, 4.75), (6.5, 5.75)), {'T'})], (15.5,
   16): [(((6.5, 5.5), (6.0, 5.0)), {'T'})]}}

In [143]:
css= sts.getCommunsequences()
css

{4: [{('person_bleu',
    'person_noir',
    'person_rouge',
    'person_vert'): {(8.5,
     9): [(((5.4375, 1.0), (6.3125, 2.0)),
      {'T'})], (14.5, 15): [(((7.875, 3.8125), (7.75, 4.5625)), {'T'})]}}],
 3: [{('person_bleu',
    'person_rouge',
    'person_vert'): {(8, 9): [(((5.375, 1.0), (6.375, 2.0)), {'T'})], (10.1,
     11): [(((7.25, 2.5), (8.0, 3.0)), {'C'})], (14.5,
     15): [(((7.875, 3.75), (7.75, 4.5)), {'T'})], (17.5,
     18): [(((7.125, 3.875), (6.375, 4.125)), {'S'})]}},
  {('person_noir',
    'person_rouge',
    'person_vert'): {(8.5, 9): [(((5.5, 1.0), (6.25, 2.0)), {'T'})], (14.5,
     15): [(((7.875, 3.875), (7.75, 4.625)), {'T'})]}},
  {('person_bleu',
    'person_noir',
    'person_vert'): {(8.5, 9): [(((4.125, 0.5), (4.875, 1.5)), {'T'})], (14.5,
     15): [(((7.75, 3.375), (8.25, 4.375)), {'T'})]}},
  {('person_bleu',
    'person_noir',
    'person_rouge'): {(8.5, 9): [(((3.375, 1.5), (4.125, 2.5)), {'T'})], (11,
     12): [(((5.25, 3.625), (6.0, 4.375)), {'

In [144]:
attributes = sts.getAttributs()
attributes

[{(8.5, 9): {'P0', 'T'}},
 {(14.5, 15): {'P0', 'T'}},
 {(8, 9): {'P0', 'T'}},
 {(10.1, 11): {'C', 'P0'}},
 {(17.5, 18): {'P0', 'S'}},
 {(11, 12): {'C', 'P0'}},
 {(13, 14): {'M', 'P0'}},
 {(10, 11): {'C', 'P0'}},
 {(16, 17): {'P0', 'S'}},
 {(14, 15): {'P0', 'T'}},
 {(17, 18): {'P0', 'S'}},
 {(9.2, 9.5): {'P0', 'T'}},
 {(10.1, 12): {'C', 'P0'}},
 {(9.5, 10): {'P0', 'T'}},
 {(15.5, 16): {'P0', 'T'}},
 {(8.5, 9): {'P44', 'T'}}]

In [146]:
context_table = sts.getContext_table()
context_table

,"(8.5, 9):{'T', 'P0'}","(14.5, 15):{'T', 'P0'}","(8, 9):{'T', 'P0'}","(10.1, 11):{'C', 'P0'}","(17.5, 18):{'P0', 'S'}","(11, 12):{'C', 'P0'}","(13, 14):{'P0', 'M'}","(10, 11):{'C', 'P0'}","(16, 17):{'P0', 'S'}","(14, 15):{'T', 'P0'}","(17, 18):{'P0', 'S'}","(9.2, 9.5):{'T', 'P0'}","(10.1, 12):{'C', 'P0'}","(9.5, 10):{'T', 'P0'}","(15.5, 16):{'T', 'P0'}","(8.5, 9):{'P44', 'T'}"
person_vert,1,1,1,1,1,0,0,1,1,1,1,0,0,0,0,0
person_rouge,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0,0
person_bleu,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,0
person_noir,1,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0
person_noir122,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
person_noir123,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [148]:
context=sts.getContext()
fca=FCAsequence(context)
concept = fca.generate_concepts()

concept_table=sts.build_data_frame_result(concept)
concept_table

,Extent,Intent
0,"{person_noir, person_rouge, person_noir123, pe...",{}
1,"{person_noir, person_rouge, person_bleu, perso...","{(8.5, 9):{'T', 'P0'}, (14.5, 15):{'T', 'P0'}}"
2,"{person_rouge, person_bleu, person_vert}","{(10.1, 11):{'C', 'P0'}, (14.5, 15):{'T', 'P0'..."
3,"{person_noir, person_rouge, person_bleu}","{(8.5, 9):{'T', 'P0'}, (14.5, 15):{'T', 'P0'},..."
4,"{person_noir123, person_noir122}","{(8.5, 9):{'P44', 'T'}}"
5,"{person_bleu, person_vert}","{(10.1, 11):{'C', 'P0'}, (10.1, 15):{ 'P0','T'..."
6,"{person_rouge, person_vert}","{(16, 17):{'P0', 'S'}, (17.5, 18):{'P0', 'S'},..."
7,"{person_rouge, person_bleu}","{(9.2, 9.5):{'T', 'P0'}, (17.5, 18):{'P0', 'S'..."
8,"{person_noir, person_bleu}","{(8.5, 9):{'T', 'P0'}, (11, 12):{'C', 'P0'}, (..."
9,{person_vert},"{(14, 15):{ 'P0','T'}, (8, 9):{ 'P0','T'}, (10..."


In [149]:
l=sts.get_centers_locations(sts.sort_result(concept))
concept_table["location"]=l
concept_table

,Extent,Intent,location
0,"{person_noir, person_rouge, person_noir123, pe...",{},None
1,"{person_noir, person_rouge, person_bleu, perso...","{(8.5, 9):{'T', 'P0'}, (14.5, 15):{'T', 'P0'}}","[{(8.5, 9): (4.125, 1.5)}, {(14.5, 15): (6.812..."
2,"{person_rouge, person_bleu, person_vert}","{(10.1, 11):{'C', 'P0'}, (14.5, 15):{'T', 'P0'...","[{(10.1, 11): (7.0, 3.1666666666666665)}, {(14..."
3,"{person_noir, person_rouge, person_bleu}","{(8.5, 9):{'T', 'P0'}, (14.5, 15):{'T', 'P0'},...","[{(8.5, 9): (2.8333333333333335, 1.66666666666..."
4,"{person_noir123, person_noir122}","{(8.5, 9):{'P44', 'T'}}","[{(8.5, 9): (90.0, 5.25)}]"
5,"{person_bleu, person_vert}","{(10.1, 11):{'C', 'P0'}, (10.1, 15):{ 'P0','T'...","[{(10.1, 11): (6.75, 2.5)}, {(10.1, 11): None,..."
6,"{person_rouge, person_vert}","{(16, 17):{'P0', 'S'}, (17.5, 18):{'P0', 'S'},...","[{(16, 17): (6.625, 5.125)}, {(17.5, 18): (5.6..."
7,"{person_rouge, person_bleu}","{(9.2, 9.5):{'T', 'P0'}, (17.5, 18):{'P0', 'S'...","[{(9.2, 9.5): (4.5, 2.625)}, {(17.5, 18): (3.7..."
8,"{person_noir, person_bleu}","{(8.5, 9):{'T', 'P0'}, (11, 12):{'C', 'P0'}, (...","[{(8.5, 9): (1.0, 1.0)}, {(11, 12): (3.75, 3.5..."
9,{person_vert},"{(14, 15):{ 'P0','T'}, (8, 9):{ 'P0','T'}, (10...","[{(14, 15): (10.0, 2.5)}, {(8, 9): (8.0, 1.0)}..."


In [171]:
dr = sts.build_data_frame_result(concept)
dr

,Extent,Intent
0,"{person_noir, person_rouge, person_noir123, pe...",{}
1,"{person_noir, person_rouge, person_bleu, perso...","{(8.5, 9):{'T', 'P0'}, (14.5, 15):{'T', 'P0'}}"
2,"{person_rouge, person_bleu, person_vert}","{(10.1, 11):{'C', 'P0'}, (14.5, 15):{'T', 'P0'..."
3,"{person_noir, person_rouge, person_bleu}","{(8.5, 9):{'T', 'P0'}, (14.5, 15):{'T', 'P0'},..."
4,"{person_noir123, person_noir122}","{(8.5, 9):{'P44', 'T'}}"
5,"{person_bleu, person_vert}","{(10.1, 11):{'C', 'P0'}, (10.1, 15):{ 'P0','T'..."
6,"{person_rouge, person_vert}","{(16, 17):{'P0', 'S'}, (17.5, 18):{'P0', 'S'},..."
7,"{person_rouge, person_bleu}","{(9.2, 9.5):{'T', 'P0'}, (17.5, 18):{'P0', 'S'..."
8,"{person_noir, person_bleu}","{(8.5, 9):{'T', 'P0'}, (11, 12):{'C', 'P0'}, (..."
9,{person_vert},"{(14, 15):{ 'P0','T'}, (8, 9):{ 'P0','T'}, (10..."


<h1>________The End________</h1>

In [154]:
# def moyenne_coordonnees(coordonnees):
#     # Initialiser les variables pour stocker la somme des coordonnées et le nombre de points
#     somme_x = 0
#     somme_y = 0
#     nombre_points = 0

#     # Parcourir chaque paire de coordonnées et chaque point
#     for paire in coordonnees:
#         for point in paire:
#             somme_x += point[0]
#             somme_y += point[1]
#             nombre_points += 1

#     # Calculer les moyennes
#     moyenne_x = somme_x / nombre_points
#     moyenne_y = somme_y / nombre_points

#     return moyenne_x, moyenne_y
# moyenne_coordonnees([((7.5, 0.5), (8.5, 1.5)), ((6, 2.5), (7, 3.5))])

In [155]:
# def get_centers_locations(data,sorted_results):
#     concepts=dict()
#     for intent, extent in sorted_results:
#         if intent:
#             concepts[tuple(extent)]=list()
#             for interval_selected in get_liste_interval(intent):
#                 liste_person_location=list()
#                 for individual in data:
#                     for interval in data[individual]:
#                         inter = intersection_intervals(interval,interval_selected)
#                         if inter and individual in extent:
#                             liste_person_location.append((inter,individual,data[individual][interval][0]))
#                 concepts[tuple(extent)].append(liste_person_location)

#         else:
#             concepts[tuple(extent)]=None  
#     person_location_centers=dict()

#     for group in concepts:
#         if concepts[group]:
#             person_location_centers[group]=list()
#             for subgroup in concepts[group]:
#                 subgroup_locations=dict()
#                 liste_locations=[]
#                 for triple in subgroup:
#                     key_time=triple[0]
#                     subgroup_locations[key_time]=None
#                     liste_locations.append(triple[2])
#                 subgroup_locations[key_time]=moyenne_coordonnees(liste_locations)
#                 person_location_centers[group].append(subgroup_locations)
#         else:
#             person_location_centers[group]=None
#     liste_time=[]
#     for group in person_location_centers:
#         liste_time.append(person_location_centers[group])
#     return liste_time

In [156]:
# loc=get_centers_locations(data,sorted_results)
# concept_table["location"]=loc
# concept_table

In [157]:
# person_location_centers=dict()

# for group in concepts:
#     if concepts[group]:
#         person_location_centers[group]=list()
#         for subgroup in concepts[group]:
#             subgroup_locations=dict()
#             liste_locations=[]
#             for triple in subgroup:
#                 key_time=triple[0]
#                 subgroup_locations[key_time]=None
#                 liste_locations.append(triple[2])
#             subgroup_locations[key_time]=moyenne_coordonnees(liste_locations)
#             person_location_centers[group].append(subgroup_locations)
# #             print(subgroup)
# #             print(group,)
# #             print("___")
#     else:
#         person_location_centers[group]=None
# liste_time=[]
# for group in person_location_centers:
#     liste_time.append(person_location_centers[group])
# concept_table["location"]=liste_time
# concept_table

In [158]:
# person_location=dict()
# liste_person_location=list()
# concepts=dict()
# for intent, extent in sorted_results:
#     if len(list(intent))!=0:
#         concepts[tuple(extent)]=list()
#         for individual in data:
#             if individual in extent:
#                 for interval in data[individual]:
#                         associeted_caracteristiques=[]
#                         for interval_selected in get_liste_interval(intent):
#                             if intersection_intervals(interval,interval_selected):
#                                 associeted_caracteristiques.append((individual,data[individual][interval],data[individual][interval][0])) 
#                         if associeted_caracteristiques:
#         concepts[tuple(extent)].append(associeted_caracteristiques)       
#     else:
#         concepts[tuple(extent)]=None
                                
# liste_groups=[]        
# for key in concepts:
#     if concepts[key]:
#         print((concepts[key]))
#         print("__")
        

In [ ]:


# import math

# def distance_entre_points(point1, point2): #distance orthodromique (ou formule de Haversine)
#     lat1, lon1=point1
#     lat2, lon2=point2
    
#     # Rayon de la Terre en kilomètres
#     r = 6371.0
    
#     # Conversion des degrés en radians
#     lat1 = math.radians(lat1)
#     lon1 = math.radians(lon1)
#     lat2 = math.radians(lat2)
#     lon2 = math.radians(lon2)
    
#     # Différences de latitude et de longitude
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
    
#     # Formule de Haversine
#     a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
#     c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
#     # Distance en metres
#     distance = r * c
#     return distance*1000
# def distance(a,b):
#     lat1, lon1=a
#     lat2, lon2=b
#     return math.sqrt((lat2-lat1)**2+(lon2-lon1)**2)
# for person in data:
#     i=0
#     for interval in data[person]:
#         for person2 in data:
#             j=0
#             for interval2 in data[person2]:
# #         print(data[person][interval][0])
# #         d=distance_entre_points(data[person][interval][0][0],data[person][interval][0][1])
#                 if person!=person2:
#                     d=distance(data[person][interval][0][0],data[person2][interval2][0][0])+distance(data[person][interval][0][1],data[person2][interval2][0][1])
#                     if d<=3:
#                         print(person,person2,i,j,"==>",d)
#                 j+=1
#         i+=1

In [159]:
# def intersection_intervals(interval1, interval2):
#     start = max(interval1[0], interval2[0])
#     end = min(interval1[1], interval2[1])
    
#     if start < end:
#         return (start, end)
#     else:
#         return None

In [160]:
# def transform_data_if_needed(inner_dict):
#     new_inner_dict = {}
    
#     for inner_key, value in inner_dict.items():
#         combined_set = set()
#         associated_tuples = []
        
#         # Normaliser la structure en une liste de tuples pour traitement uniforme
#         values_to_process = value if isinstance(value, list) else [value]
        
#         for item in values_to_process:
#             if isinstance(item, tuple) and len(item) == 2:
#                 associated_tuples.append(item[0])
#                 combined_set.update(item[1])
        
#         new_inner_dict[inner_key] = (tuple(associated_tuples[0]), combined_set)
    
#     return new_inner_dict




# # test example errrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
# inner_data_1 = {(8.5, 9): [(((0.75, 0.5), (1.25, 1.5)), {'T'})], (11,
#    12): [(((3.0, 3.25), (4.5, 3.75)), {'C'})], (14.5,
#    15): [(((5.5, 4.75), (6.5, 5.75)), {'T'})]}

# transformed_inner_data_1 = transform_data_if_needed(inner_data_1)
# print(transformed_inner_data_1)

# inner_data_2 = {(8.5, 9): (((0.75, 0.5), (1.25, 1.5)), {'T'}), (11, 12): (((3.0, 3.25), (4.5, 3.75)), {'C'}), (14.5, 15): (((5.5, 4.75), (6.5, 5.75)), {'T'})}

# transformed_inner_data_2 = transform_data_if_needed(inner_data_2)
# print(transformed_inner_data_2)


In [296]:
# passed_keys=[]
# # matched_person=set()
# cardinality=len(list(data.keys()))

# def commun_sequences(sequence1, sequence2,id1,id2=list,treshold_meter=3):
    
#     sequence2 = transform_data_if_needed(sequence2)
#     ids={id1,*id2}
#     idsl=list(ids)
#     idsl.sort()
#     matched_person={tuple(idsl):{}}
#     sequence_commun={}
#     for interval1 in sequence1:
#         for interval2 in sequence2:
#             if intersection_intervals(interval1, interval2):
# #                 print(sequence1[interval1][1])
#                 if (distance(sequence1[interval1][0][0],sequence2[interval2][0][0])+distance(sequence1[interval1][0][1],sequence2[interval2][0][1]))<=treshold_meter:
# #                     print(sequence1[interval1][1])
#                     if sequence1[interval1][1] & sequence2[interval2][1]: #ici le travail
#                         if len(list(sequence1[interval1][1] & sequence2[interval2][1]))>1:
#                             sequence_commun[intersection_intervals(interval1, interval2)]=list()
#                             for elm in list(sequence1[interval1][1] & sequence2[interval2][1]):
#                                 sequence_commun[intersection_intervals(interval1, interval2)].append(
#                                                                                                         ( ( ((sequence1[interval1][0][0][0]+sequence2[interval2][0][0][0])/2,(sequence1[interval1][0][0][1]+sequence2[interval2][0][0][1])/2),
#                                                                                                             (((sequence1[interval1][0][1][0]+sequence2[interval2][0][1][0])/2,(sequence1[interval1][0][1][1]+sequence2[interval2][0][1][1])/2))   
#                                                                                                           ),
#                                                                                                           {elm}
#                                                                                                         )
#                                                                                                     )         
#                         else:
#                             sequence_commun[intersection_intervals(interval1, interval2)]=[( ( ((sequence1[interval1][0][0][0]+sequence2[interval2][0][0][0])/2,(sequence1[interval1][0][0][1]+sequence2[interval2][0][0][1])/2),
#                                                                                                             (((sequence1[interval1][0][1][0]+sequence2[interval2][0][1][0])/2,(sequence1[interval1][0][1][1]+sequence2[interval2][0][1][1])/2))   
#                                                                                                           ),
#                                                                                                           sequence1[interval1][1] & sequence2[interval2][1]
#                                                                                                         )]   
#                         idsl=list(ids)
#                         idsl.sort()
#                         matched_person[tuple(idsl)]= sequence_commun
#     if len(list(matched_person[tuple(idsl)])) ==0:
#         return None
#     else:
#         return matched_person
# commun_sequences(data["person_bleu"],data["person_noir"],"person_bleu",["person_noir"],treshold_meter=2)


In [161]:
# from itertools import chain, combinations

# def powerset(iterable): 
#     s = list(iterable)         
#     return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
# def getGroupsMore_N_element(data,cardinality):#pour creer des combinison des cles de sequence
#     groups={}
#     for i in range(cardinality,len(list(data.keys()))+1): #initialisation de dictionnaire des combinaisons (cle:[])
#         groups[len(list(data.keys()))+cardinality-i]=list() # sachant que la cle represent la cardinalite des groupes
#                                           # c-a-d les groupes des listes de 2 elements de 3,4 etc
    
#     for obj in powerset(list(data.keys())):
#         if len(list(obj))>=cardinality:
#             groups[len(list(obj))].append(set(obj))
#     return groups

In [162]:
# groupes=getGroupsMore_N_element(data,cardinality=2)
# groupes

In [163]:
# def getCommunsequences(groups,data,treshold=0):
#     results={}
#     for keyGroup in groups:
#         results[keyGroup]=list()
        
#     for keyGroup in groups: 
#         for group in groups[keyGroup]:
#             liste=list(group)
#             liste.sort()
#             result=None
#             result =commun_sequences(data[liste[0]],data[liste[1]],liste[0],[liste[1]],treshold_meter=treshold) #common subsequence between first and second sequence           
#             for i in range(1,len(liste)):
#                 if result!=None and liste[i] not in list(list(result)[0]): # le 2eme condi pour le cas de repetition de comparaison 
# #                     print(list(result)[0],liste[i])
#                     result =commun_sequences(data[liste[i]],result[list(result)[0]],liste[i],list(list(result)[0]),treshold_meter=treshold)
#                     if result==None:
#                         break
#             if result :
#                 results[keyGroup].append(result)
                
#     deleted_keys=[]
#     for key in results:
#         if len(results[key])==0:
#             deleted_keys.append(key)
#     for key in deleted_keys:
#         del results[key]
#     return results



# resultats = getCommunsequences(groupes,data,treshold=5)
# for cardinality in resultats:
#     print("les groupes qui ont des sous-sequence communes de cardinalite : ",cardinality)
#     print("_____")
#     for group in resultats[cardinality]:
#         print(group)
#     print("______________/\/\/\/\/\/\/\/\/\___________________________")

In [164]:
# memberes = list(data.keys())
# memberes

In [165]:
# def dropLocationPart(clusters):
#     new_clusters=dict()
#     for cardinality in clusters:
#         new_clusters[cardinality]=list()
#         group_key=dict()
# #         print(cardinality)
#         for group in clusters[cardinality]:
#             group_key[list(group.keys())[0]]=dict()
# #             print(list(group.keys())[0])
#             for key in group:
#                 for interval_time in group[key]:
#                     for item in group[key][interval_time]:
#                         group_key[key][interval_time]=[item[1]]
#         new_clusters[cardinality].append(group_key)
#     return new_clusters
                    
# #                     print(group[key][interval_time])
            
# #                 print(group[key])
# # print(resultats[2])
# resultat_dropped_location=dropLocationPart(resultats)
# resultat_dropped_location

In [166]:
# def getAttributs(clusters): #extract attributs from clusters of groups and commun subsequences to
#     attributes=[]               #create context
#     for cardinality in clusters:
#         for group in clusters[cardinality]:
#             for key in group:
#                 for item in group[key]:
# #                     print(item,"-",group[key][item])
# #                     gk = list(item)[0] # time interval
# #                     if {gk:group[key][gk][0]} not in attributes:
# #                     if {item:group[key][item]} not in attributes:
#                     if len(group[key][item])>1: #cette condition peux creer des repititions on va les elminer avec un autre condition        
#                         for el in group[key][item]:
#                             if {item:el} not in attributes:
#                                 attributes.append({item:el})
#                     else:
#                         if {item:group[key][item][0]} not in attributes:
#                             attributes.append({item:group[key][item][0]})

#     return attributes
# caracteristiques = getAttributs(resultat_dropped_location)
# caracteristiques

In [167]:
# import pandas as pd
# def getContext_table(resultat_dropped_location,caracteristiques):
#     resultats=resultat_dropped_location
#     columns=[]
#     for caracteristique in caracteristiques:
#         for item in caracteristique:
#             columns.append(f"{item}:{caracteristique[item]}")
#     df = pd.DataFrame(0, index=memberes, columns=columns)

#     # Remplir le DataFrame
#     for cardi in resultats:
#         for item in resultats[cardi]:
#             for groupe, caract in item.items():
#                 for member in groupe:
#                     for car in caract:
#     #                     print(f"{car}:{caract[car]}")
#                         if len(caract[car])>=2:
#                             for e in caract[car]:
#                                 print(f"{car}:{e,member}")
#                                 col_name = f"{car}:{e}"
#     #                             print(col_name)
#                                 df.at[member, col_name] = 1
#                         else:
#                             col_name = f"{car}:{caract[car][0]}"
#                             df.at[member, col_name] = 1
#     return df
#     #                         print(caract[car][0])
# df1=getContext_table(resultat_dropped_location,caracteristiques)
# df1.head(7)

In [168]:
# def getContext(Binary_dataframe=pd.DataFrame()):
#     context = dict()
#     for index,row in Binary_dataframe.iterrows():
#         attributs = []
#         for i in range(len(row)):
#             if row[i]==1:
#                 attributs.append(list(Binary_dataframe.columns)[i])
#         context[index]=set(attributs)    
#     return context
    
# context = getContext(df)
# context

In [169]:
# fca=FCAsequence(context)
# concept = fca.generate_concepts()

# # Créer une liste pour stocker les résultats
# results = []

# for extent, intent in concept:
#     results.append((set(intent), set(extent)))

# # Trier les résultats par le nombre d'individus (taille de l'ensemble d'individus) en ordre décroissant
# sorted_results = sorted(results, key=lambda x: len(x[1]), reverse=True)

# intents=[]
# extents=[]
# # # Afficher les résultats triés
# # for intent, extent in sorted_results:
# #     print(f"Predicats: {intent} \n Individuals: {extent}\n")
# for intent, extent in sorted_results:
#     intents.append(intent)
#     extents.append(extent)
# concept_table=pd.DataFrame()
# concept_table["Extent"]=extents
# concept_table["Intent"]=intents
# concept_table.head()

In [170]:
# import folium
# from geopy.distance import geodesic
# import numpy as np

# # Exemple de coordonnées (latitude, longitude)
# coordinates = [
#     (48.8566, 2.3522),  # Paris
#     (51.5074, -0.1278), # Londres
#     (40.7128, -74.0060),# New York
#     (34.0522, -118.2437)# Los Angeles
# ]

# # Calcul du centre de gravité
# lats, lons = zip(*coordinates)
# center_lat = np.mean(lats)
# center_lon = np.mean(lons)
# center_of_gravity = (center_lat, center_lon)

# # Calcul du rayon du cercle
# max_distance = max(geodesic(center_of_gravity, coord).kilometers for coord in coordinates)

# # Création de la carte centrée sur le centre de gravité
# m = folium.Map(location=center_of_gravity, zoom_start=2)

# # Ajout des marqueurs pour les coordonnées
# for coord in coordinates:
#     folium.Marker(location=coord).add_to(m)

# # Ajout du marqueur pour le centre de gravité
# folium.Marker(location=center_of_gravity, popup='Centre de Gravité', icon=folium.Icon(color='red')).add_to(m)

# # Ajout du cercle couvrant
# folium.Circle(
#     location=center_of_gravity,
#     radius=max_distance * 1000,  # convertir en mètres
#     color='green',
#     fill=True,
#     fill_opacity=0.2
# ).add_to(m)

# # Enregistrer la carte dans un fichier HTML
# m.save('map.html')

# # Afficher la carte
# m


In [391]:
# from jinja2 import Template
# from IPython.core.display import display, HTML

# # Définir le template HTML
# template_html = """
# <!DOCTYPE html>
# <html lang="en">
# <head>
#     <meta charset="UTF-8">
#     <title>Table Example</title>
#     <style>
#         body {
#             font-family: Arial, sans-serif;
#         }
#         .container {
#             width: 80%;
#             margin: auto;
#             padding: 20px;
#         }
#         table {
#             width: 100%;
#             border-collapse: collapse;
#             margin-top: 20px;
#         }
#         th, td {
#             border: 1px solid #ddd;
#             padding: 8px;
#             text-align: left;
#         }
#         th {
#             background-color: #f2f2f2;
#         }
#     </style>
# </head>
# <body>
#     <div class="container">
#         <h1>{{ title }}</h1>
#         <table>
#             <thead>
#                 <tr>
#                     <th>Column 1</th>
#                     <th>Column 2</th>
#                     <th>Column 3</th>
#                 </tr>
#             </thead>
#             <tbody>
#                 {% for row in rows %}
#                 <tr>
#                     <td>{{ row[0] }}</td>
#                     <td>{{ row[1] }}</td>
#                     <td>{{ row[2] }}</td>
#                 </tr>
#                 {% endfor %}
#             </tbody>
#         </table>
#     </div>
# </body>
# </html>
# """

# # Définir les données de la table
# data = {
#     "title": "Exemple de Table",
#     "rows": [
#         ["Ligne 1, Col 1ddddddddddddddddddddddd dddddddddddddddddddddddddddddddddddddd dddddddddddddddddddddddddddd", "Ligne 1, Col 2", "Ligne 1, Col 3"],
#         ["Ligne 2, Col 1", "Ligne 2, Col 2", "Ligne 2, Col 3"],
#         ["Ligne 3, Col 1", "Ligne 3, Col 2", "Ligne 3, Col 3"],
#         ["Ligne 4, Col 1", "Ligne 4, Col 2", "Ligne 4, Col 3"],
#         ["Ligne 5, Col 1", "Ligne 5, Col 2", "Ligne 5, Col 3"],
#     ]
# }

# # Créer un objet template Jinja2
# template = Template(template_html)

# # Rendre le template avec les données
# html_content = template.render(data)

# # Afficher le contenu HTML dans Jupyter Notebook
# display(HTML(html_content))
